In [1]:
import numpy as np
import pandas as pd
import os

In [8]:
data_directory = "../Data/"
hololenses = ['A/', 'B/']
intermediate = ''
#subdirectory = "recalibrated_ssHeadConstrained/"

In [3]:
def clean_df(df):
    result = df.loc[(df['Movement'] != "start") & (df['Movement'] != "transition") & (df['EyeTrackingEnabled'] == True) & (df['EyeCalibrationValid'] == True) & (df['EyeTrackingEnabledAndValid'] == True) & (df['EyeTrackingDataValid'] == True) & (df['GazeInputSource'] == "Eyes")].copy()
    #result = result[result['Movement'] != "transition"]
    #result = result[result['EyeTrackingDataValid'] == 'TRUE']
    #result = result[result['GazeInputSource'] == 'Eyes']
    result.reset_index(inplace=True, drop=True)
    return result

In [10]:
def calculate_cosine_error(df):
    df['cosineError'] = df.apply(lambda row: np.degrees(np.arccos(np.dot(np.array((row['localEyeGazeX'], row['localEyeGazeY'], row['localEyeGazeZ'])), np.array((row['localTransform.position.x'], row['localTransform.position.y'], row['localTransform.position.z'])))/(np.linalg.norm(np.array((row['EyeGazeDirection.x'], row['EyeGazeDirection.y'], row['EyeGazeDirection.z']))) * np.linalg.norm(np.array((row['localTransform.position.x'], row['localTransform.position.y'], row['localTransform.position.z'])))))), axis=1)
    return df

In [5]:
def calculate_euclidean_error(df):
    #df['euclideanError'] = df.apply(lambda row: np.linalg.norm(np.array((row['GazeAngleX'], row['GazeAngleY']))-np.array((row['TargetAngleX'], row['TargetAngleY']))), axis=1)
    #df['recalibratedEuclideanError'] = df.apply(lambda row: np.linalg.norm(np.array((row['CalibratedGazeAngleX'], row['CalibratedGazeAngleY']))-np.array((row['TargetAngleX'], row['TargetAngleY']))), axis=1)
    df['euclideanError'] = np.sqrt((df.GazeAngleX-df.TargetAngleX) * (df.GazeAngleX-df.TargetAngleX) + (df.GazeAngleY-df.TargetAngleY) * (df.GazeAngleY-df.TargetAngleY))
    df['recalibratedEuclideanError'] = np.sqrt((df.CalibratedGazeAngleX-df.TargetAngleX) * (df.CalibratedGazeAngleX-df.TargetAngleX) + (df.CalibratedGazeAngleY-df.TargetAngleY) * (df.CalibratedGazeAngleY-df.TargetAngleY))
    return df

In [6]:
def calculate_errors(subdirectory):
    output_subdirectory = 'error/' + subdirectory
    for hololens in hololenses:
        participantsList = os.listdir(data_directory + hololens)
        for participant in participantsList:
            print(participant)
            if not participant == '.DS_Store':
                if os.path.exists(data_directory + hololens + participant + '/' + intermediate + output_subdirectory) and len(os.listdir(data_directory + hololens + participant + '/' + intermediate + output_subdirectory)) == 12:
                    print('already calculated')
                    continue
                fileList = os.listdir(data_directory + hololens + participant + '/' + intermediate + subdirectory)
                for file in fileList:
                    if '.csv' in file:
                        filePath = data_directory + hololens + participant + '/' + intermediate
                        df = pd.read_csv(filePath + subdirectory + file)
                        error_df = calculate_euclidean_error(df)
                        error_df = calculate_cosine_error(error_df)
                        if not os.path.exists(filePath + output_subdirectory):
                            os.makedirs(filePath + output_subdirectory)
                        error_df_output_path = filePath + output_subdirectory + '/' + file
                        #error_df.to_csv(error_df_output_path, index=False)
                        print(file)
                        print("uncalibrated: " + str(error_df['euclideanError'].mean()))
                        print("calibrated: " + str(error_df['recalibratedEuclideanError'].mean()))
                        print("cosine: " + str(error_df['cosineError'].mean()))
                        print("")

In [11]:
directories = ['recalibrated_calibration/', 'recalibrated_ssHeadConstrained/', 'recalibrated_wsBodyConstrained/', 'recalibrated_ssWalking/', 'recalibrated_wsWalking/', 'recalibrated_hallway/']
for directory in directories:
    print("")
    print(directory)
    calculate_errors(directory)
# calculate_errors('recalibrated_calibration/')


recalibrated_calibration/
03
ssWalking_20230505113947_shifted0.csv
uncalibrated: 5.3615589399474946
calibrated: 5.359497388788248
cosine: 5.30097761380738

hallway_20230505114146_shifted0.csv
uncalibrated: 5.9789941699485585
calibrated: 5.709982784859909
cosine: 5.952939205368851

hallway_20230505112428_shifted0.csv
uncalibrated: 5.8314926589943115
calibrated: 5.6045340227452565
cosine: 5.799862101268502

wsBodyConstrained_20230505111519_shifted0.csv
uncalibrated: 3.4806171976889053
calibrated: 3.234143150604176
cosine: 3.4594924126441486

ssHeadConstrained_20230505111305_shifted0.csv
uncalibrated: 2.4033883483839875
calibrated: 2.311470221724084
cosine: 2.3694884657303765

ssWalking_20230505112139_shifted10.csv
uncalibrated: 5.475652907413207
calibrated: 5.477451438763881
cosine: 5.421546424291178

calibration_20230505112000_shifted0.csv
uncalibrated: 0.7145541577052661
calibrated: 0.42414492961184835
cosine: 0.7008222532064684

wsWalking_20230505113625_shifted28.csv
uncalibrated: 10

KeyboardInterrupt: 